<a href="https://colab.research.google.com/github/muranja/Wan2.1/blob/main/Wan2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Wan2.1 I2V Video Generation - Google Colab
# Fast 4 steps Wan 2.1 I2V (14B) with CausVid LoRA

# ===== SETUP AND INSTALLATION =====
# Run this cell first to install all dependencies

import subprocess
import sys

def install_requirements():
    """Install all required packages"""
    print("🔧 Installing requirements...")

    # Install specific diffusers commit
    subprocess.run([
        sys.executable, "-m", "pip", "install",
        "git+https://github.com/huggingface/diffusers.git@3a23d941f559759195dd30b5d206008f9e34f2bb"
    ], check=True)

    # Install other requirements
    requirements = [
        "transformers",
        "accelerate",
        "safetensors",
        "sentencepiece",
        "peft",
        "ftfy",
        "imageio-ffmpeg",
        "opencv-python",
        "gradio"
    ]

    for req in requirements:
        subprocess.run([sys.executable, "-m", "pip", "install", req], check=True)

    print("✅ All requirements installed!")

# Uncomment and run this line on first execution
# install_requirements()

# ===== IMPORTS =====
import torch
from diffusers import AutoencoderKLWan, WanImageToVideoPipeline, UniPCMultistepScheduler
from diffusers.utils import export_to_video
from transformers import CLIPVisionModel
import gradio as gr
import tempfile
from huggingface_hub import hf_hub_download
import numpy as np
from PIL import Image
import random
import os

# ===== MODEL CONFIGURATION =====
MODEL_ID = "Wan-AI/Wan2.1-I2V-14B-480P-Diffusers"
LORA_REPO_ID = "Kijai/WanVideo_comfy"
LORA_FILENAME = "Wan21_CausVid_14B_T2V_lora_rank32.safetensors"

# ===== CONSTANTS =====
MOD_VALUE = 32
DEFAULT_H_SLIDER_VALUE = 512
DEFAULT_W_SLIDER_VALUE = 896
NEW_FORMULA_MAX_AREA = 480.0 * 832.0

SLIDER_MIN_H, SLIDER_MAX_H = 128, 896
SLIDER_MIN_W, SLIDER_MAX_W = 128, 896
MAX_SEED = np.iinfo(np.int32).max

FIXED_FPS = 24
MIN_FRAMES_MODEL = 8
MAX_FRAMES_MODEL = 81

default_prompt_i2v = "make this image come alive, cinematic motion, smooth animation"
default_negative_prompt = "Bright tones, overexposed, static, blurred details, subtitles, style, works, paintings, images, static, overall gray, worst quality, low quality, JPEG compression residue, ugly, incomplete, extra fingers, poorly drawn hands, poorly drawn faces, deformed, disfigured, misshapen limbs, fused fingers, still picture, messy background, three legs, many people in the background, walking backwards, watermark, text, signature"

# ===== MODEL LOADING =====
def load_model():
    """Load and configure the model pipeline"""
    print("🔄 Loading model components...")

    # Check if CUDA is available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    if device == "cpu":
        print("⚠️  Warning: CUDA not available, using CPU. This will be very slow!")

    # Load model components
    image_encoder = CLIPVisionModel.from_pretrained(
        MODEL_ID,
        subfolder="image_encoder",
        torch_dtype=torch.float32
    )

    vae = AutoencoderKLWan.from_pretrained(
        MODEL_ID,
        subfolder="vae",
        torch_dtype=torch.float32
    )

    pipe = WanImageToVideoPipeline.from_pretrained(
        MODEL_ID,
        vae=vae,
        image_encoder=image_encoder,
        torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32
    )

    # Configure scheduler
    pipe.scheduler = UniPCMultistepScheduler.from_config(
        pipe.scheduler.config,
        flow_shift=8.0
    )

    pipe.to(device)

    # Download and load LoRA
    print("📥 Downloading LoRA weights...")
    causvid_path = hf_hub_download(repo_id=LORA_REPO_ID, filename=LORA_FILENAME)
    pipe.load_lora_weights(causvid_path, adapter_name="causvid_lora")
    pipe.set_adapters(["causvid_lora"], adapter_weights=[0.95])
    pipe.fuse_lora()

    print("✅ Model loaded successfully!")
    return pipe

# Load the model
pipe = load_model()

# ===== UTILITY FUNCTIONS =====
def _calculate_new_dimensions_wan(pil_image, mod_val, calculation_max_area,
                                 min_slider_h, max_slider_h,
                                 min_slider_w, max_slider_w,
                                 default_h, default_w):
    """Calculate optimal dimensions for the input image"""
    orig_w, orig_h = pil_image.size
    if orig_w <= 0 or orig_h <= 0:
        return default_h, default_w

    aspect_ratio = orig_h / orig_w

    calc_h = round(np.sqrt(calculation_max_area * aspect_ratio))
    calc_w = round(np.sqrt(calculation_max_area / aspect_ratio))

    calc_h = max(mod_val, (calc_h // mod_val) * mod_val)
    calc_w = max(mod_val, (calc_w // mod_val) * mod_val)

    new_h = int(np.clip(calc_h, min_slider_h, (max_slider_h // mod_val) * mod_val))
    new_w = int(np.clip(calc_w, min_slider_w, (max_slider_w // mod_val) * mod_val))

    return new_h, new_w

def handle_image_upload_for_dims_wan(uploaded_pil_image, current_h_val, current_w_val):
    """Handle image upload and calculate dimensions"""
    if uploaded_pil_image is None:
        return gr.update(value=DEFAULT_H_SLIDER_VALUE), gr.update(value=DEFAULT_W_SLIDER_VALUE)
    try:
        new_h, new_w = _calculate_new_dimensions_wan(
            uploaded_pil_image, MOD_VALUE, NEW_FORMULA_MAX_AREA,
            SLIDER_MIN_H, SLIDER_MAX_H, SLIDER_MIN_W, SLIDER_MAX_W,
            DEFAULT_H_SLIDER_VALUE, DEFAULT_W_SLIDER_VALUE
        )
        return gr.update(value=new_h), gr.update(value=new_w)
    except Exception as e:
        print(f"Error calculating dimensions: {e}")
        return gr.update(value=DEFAULT_H_SLIDER_VALUE), gr.update(value=DEFAULT_W_SLIDER_VALUE)

# ===== MAIN GENERATION FUNCTION =====
def generate_video(input_image, prompt, height, width,
                   negative_prompt=default_negative_prompt, duration_seconds = 2,
                   guidance_scale = 1, steps = 4,
                   seed = 42, randomize_seed = False,
                   progress=gr.Progress(track_tqdm=True)):
    """Generate video from input image and prompt"""

    if input_image is None:
        raise gr.Error("Please upload an input image.")

    target_h = max(MOD_VALUE, (int(height) // MOD_VALUE) * MOD_VALUE)
    target_w = max(MOD_VALUE, (int(width) // MOD_VALUE) * MOD_VALUE)

    num_frames = np.clip(int(round(duration_seconds * FIXED_FPS)), MIN_FRAMES_MODEL, MAX_FRAMES_MODEL)

    current_seed = random.randint(0, MAX_SEED) if randomize_seed else int(seed)

    resized_image = input_image.resize((target_w, target_h))

    print(f"🎬 Generating video with {num_frames} frames at {target_w}x{target_h}")
    print(f"🎯 Prompt: {prompt}")
    print(f"🌱 Seed: {current_seed}")

    with torch.inference_mode():
        output_frames_list = pipe(
            image=resized_image,
            prompt=prompt,
            negative_prompt=negative_prompt,
            height=target_h,
            width=target_w,
            num_frames=num_frames,
            guidance_scale=float(guidance_scale),
            num_inference_steps=int(steps),
            generator=torch.Generator(device=pipe.device).manual_seed(current_seed)
        ).frames[0]

    with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tmpfile:
        video_path = tmpfile.name

    export_to_video(output_frames_list, video_path, fps=FIXED_FPS)
    print(f"✅ Video saved to: {video_path}")
    return video_path, current_seed

# ===== GRADIO INTERFACE =====
def create_interface():
    """Create the Gradio interface"""
    with gr.Blocks(title="Wan2.1 I2V Video Generation") as demo:
        gr.Markdown("# 🎥💨 Fast 4 steps Wan 2.1 I2V (14B) with CausVid LoRA")
        gr.Markdown(
            "[CausVid](https://github.com/tianweiy/CausVid) is a distilled version of Wan 2.1 to run faster in just 4-8 steps, "
            "[extracted as LoRA by Kijai](https://huggingface.co/Kijai/WanVideo_comfy/blob/main/Wan21_CausVid_14B_T2V_lora_rank32.safetensors) "
            "and is compatible with 🧨 diffusers"
        )

        with gr.Row():
            with gr.Column():
                input_image_component = gr.Image(
                    type="pil",
                    label="Input Image (auto-resized to target H/W)"
                )
                prompt_input = gr.Textbox(
                    label="Prompt",
                    value=default_prompt_i2v
                )
                duration_seconds_input = gr.Slider(
                    minimum=round(MIN_FRAMES_MODEL/FIXED_FPS,1),
                    maximum=round(MAX_FRAMES_MODEL/FIXED_FPS,1),
                    step=0.1,
                    value=2,
                    label="Duration (seconds)",
                    info=f"Clamped to model's {MIN_FRAMES_MODEL}-{MAX_FRAMES_MODEL} frames at {FIXED_FPS}fps."
                )

                with gr.Accordion("Advanced Settings", open=False):
                    negative_prompt_input = gr.Textbox(
                        label="Negative Prompt",
                        value=default_negative_prompt,
                        lines=3
                    )
                    seed_input = gr.Slider(
                        label="Seed",
                        minimum=0,
                        maximum=MAX_SEED,
                        step=1,
                        value=42,
                        interactive=True
                    )
                    randomize_seed_checkbox = gr.Checkbox(
                        label="Randomize seed",
                        value=True,
                        interactive=True
                    )
                    with gr.Row():
                        height_input = gr.Slider(
                            minimum=SLIDER_MIN_H,
                            maximum=SLIDER_MAX_H,
                            step=MOD_VALUE,
                            value=DEFAULT_H_SLIDER_VALUE,
                            label=f"Output Height (multiple of {MOD_VALUE})"
                        )
                        width_input = gr.Slider(
                            minimum=SLIDER_MIN_W,
                            maximum=SLIDER_MAX_W,
                            step=MOD_VALUE,
                            value=DEFAULT_W_SLIDER_VALUE,
                            label=f"Output Width (multiple of {MOD_VALUE})"
                        )
                    steps_slider = gr.Slider(
                        minimum=1,
                        maximum=30,
                        step=1,
                        value=4,
                        label="Inference Steps"
                    )
                    guidance_scale_input = gr.Slider(
                        minimum=0.0,
                        maximum=20.0,
                        step=0.5,
                        value=1.0,
                        label="Guidance Scale"
                    )

                generate_button = gr.Button("Generate Video", variant="primary")

            with gr.Column():
                video_output = gr.Video(
                    label="Generated Video",
                    autoplay=True,
                    interactive=False
                )
                seed_output = gr.Number(label="Used Seed", interactive=False)

        # Event handlers
        input_image_component.upload(
            fn=handle_image_upload_for_dims_wan,
            inputs=[input_image_component, height_input, width_input],
            outputs=[height_input, width_input]
        )

        input_image_component.clear(
            fn=handle_image_upload_for_dims_wan,
            inputs=[input_image_component, height_input, width_input],
            outputs=[height_input, width_input]
        )

        ui_inputs = [
            input_image_component, prompt_input, height_input, width_input,
            negative_prompt_input, duration_seconds_input,
            guidance_scale_input, steps_slider, seed_input, randomize_seed_checkbox
        ]

        generate_button.click(
            fn=generate_video,
            inputs=ui_inputs,
            outputs=[video_output, seed_output]
        )

    return demo

# ===== LAUNCH INTERFACE =====
# Create and launch the interface
demo = create_interface()

# Launch with public URL for Colab
print("🚀 Starting Gradio interface...")
demo.launch(
    share=True,  # Creates a public URL
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

# ===== USAGE INSTRUCTIONS =====
print("""
🎉 Setup Complete!

📋 Instructions:
1. Upload an image in the interface above
2. Enter a prompt describing the motion you want
3. Adjust duration, dimensions, and other settings as needed
4. Click "Generate Video" to create your video

💡 Tips:
- Use descriptive prompts for better results
- Keep duration reasonable (2-4 seconds for best quality)
- The model works best with clear, well-lit images
- Try different seeds for varied results

⚡ Performance Notes:
- First generation may take longer due to model loading
- GPU acceleration significantly improves speed
- Reduce steps/duration if generation is too slow
""")

🔄 Loading model components...
Using device: cpu
⚠️  Warning: CUDA not available, using CPU. This will be very slow!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

...

(…)pytorch_model-00009-of-00014.safetensors:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

(…)pytorch_model-00010-of-00014.safetensors:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

(…)pytorch_model-00011-of-00014.safetensors:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

(…)pytorch_model-00012-of-00014.safetensors:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

KeyboardInterrupt: 